In [1]:
from __future__ import unicode_literals

In [2]:
import sys

sys.path.append('Perfomer_local/keras_layer_normalization')
sys.path.append('Perfomer_local/keras_performer')
sys.path.append('Perfomer_local/keras_position_wise_feed_forward')
sys.path.append('Perfomer_local/tensorflow_fast_attention')

'''
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_layer_normalization')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_performer')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_position_wise_feed_forward')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\tensorflow_fast_attention')
'''

"\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_layer_normalization')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_performer')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_position_wise_feed_forward')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\tensorflow_fast_attention')\n"

In [3]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performer as tfr
import nltk
#nltk.download('punkt') 

from os import listdir
from os.path import isfile, isdir, join

from nltk.tokenize import wordpunct_tokenize
nltk.download('punkt') #run this in first time
from nltk.tokenize import RegexpTokenizer

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
'''
def solve_cudnn_error():
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
            
solve_cudnn_error()
'''

'\ndef solve_cudnn_error():\n    import tensorflow as tf\n    gpus = tf.config.experimental.list_physical_devices(\'GPU\')\n    if gpus:\n        try:\n            # Currently, memory growth needs to be the same across GPUs\n            for gpu in gpus:\n                tf.config.experimental.set_memory_growth(gpu, True)\n            logical_gpus = tf.config.experimental.list_logical_devices(\'GPU\')\n            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n        except RuntimeError as e:\n            # Memory growth must be set before GPUs have been initialized\n            print(e)\n            \nsolve_cudnn_error()\n'

In [5]:
import csv
def readCSV(file_name):
  errlist=[]
  LBlist=[]
  with open(file_name, newline='') as csvfile:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary

    rows = csv.DictReader(csvfile)
    for row in rows: 
      RL=list(row.values())

      RL = list(map(int, RL))
      errs=RL[1:37]
      LB=RL[37:]
      errlist.append(errs)
      LBlist.append(LB)
  return errlist,LBlist

In [6]:
def parseSentence(x):	
	tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
	tokens=[]
	state="START"
	chrs=""
	for i in range(len(x)):
		#print(ord(x[i]))
		if (ord(x[i])>255):
			inp="U"
		else:
			inp="E"
	
		if state=="START":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
			
		elif state=="ASCII":
			if inp=="E":
				chrs += x[i]
			else:#U
				state="UNICODE"
				tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
				chrs=""
				tokens.append(x[i])
	
		elif state=="UNICODE":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
	if len(chrs)>0:
		tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
	return tokens

In [7]:
def readcode(fname):
    with open(fname) as f:
        data = f.read()
        return data

In [8]:
class TestTranslate(unittest.TestCase):
 
    @staticmethod
    def _build_token_dict(token_list):
        token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
        }
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
 
    def test_translate(self):
        print("i am here: " )
        source_tokens = []
        target_errors=[]
        target_LB=[]
        #source_file=[]
        
        Input_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\Trianing\InputTxt"
        Location_Output_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\Trianing\InputCSV"
        '''
        Input_Path = "Performer/Perfomer_local/Trianing/InputTxt"
        Location_Output_Path = "Performer/Perfomer_local/Trianing/InputCSV"
        '''
 
        InputFiles = sorted(listdir(Input_Path))
        OutputFiles = sorted(listdir(Location_Output_Path))
        #max_files =10
        for f in InputFiles:
          fullpath = join(Input_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            source_tokens.append(parseSentence(readcode(fullpath)))
            #if len(source_tokens)>max_files: break
        for f in OutputFiles:
          fullpath = join(Location_Output_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            err,lb = readCSV(fullpath)
            target_errors.append(err)
            target_LB.append(lb)
            #if len(source_tokens)>max_files: break
        dd=np.asarray(target_errors)
        print("AAAA: ", dd.shape)
        print("aaaa: " , type(target_errors[0][0]))
        target_errors=target_errors[0]  
        target_LB=target_LB[0]     
        print("XXXX: " , len(source_tokens))
        print("YYYY: " , type(target_errors[0][0]))
        print("ZZZZ: " , len(target_LB))

 
        source_tokens2 = []
        target_errors2= []
        target_LB2=[]

        THRESHOLD_FILE_LEN = 64000000
 
        for i in range(len(source_tokens)): 
          #print (i)
          src = source_tokens[i]
          err = target_errors[i]   
          lb = target_LB[i]                   
          if (len(src)<=THRESHOLD_FILE_LEN):
            source_tokens2.append(src)
            target_errors2.append(err)
            target_LB2.append(lb) 
        source_tokens = source_tokens2
        target_errors= target_errors2        
        target_LB= target_LB2        

        print("XXXX2: " , len(source_tokens))
                    
 
        # Generate dictionaries
        source_token_dict = self._build_token_dict(source_tokens)
        
 
        # Add special tokens
        encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
        
        #output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]
 
        # Padding
        sl = max(map(len, encode_tokens))
        
        source_max_len = sl
        
 
        encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
        
        
 
        encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
       
        
        #print("encode_input:", encode_input)
        
        #print("len(source_token_dict):", len(source_token_dict))
        
        
        #decode_output_one_hot_encoded = to_categorical(decode_output)
        #print("one-hot-decode_output:", decode_output_one_hot_encoded)
        #decode_output =  list(decode_output_one_hot_encoded)
        token_num =len(source_token_dict)
        print(type(token_num))
         # Build & fit model
        model = tfr.get_model(
            max_input_len= (source_max_len),
            errNum=36,
            lbNum=192,
            token_num=len(source_token_dict),
            embed_dim=32,
            encoder_num=4, #2
            head_num=4,
            hidden_dim=128,
            dropout_rate=0.05
        )
        losses = {    
            "error_feed_forward_output1": "categorical_crossentropy",
            "error_feed_forward_output2": "mean_squared_error",            
        }
        lossWeights = {"error_feed_forward_output1": 1.0,"error_feed_forward_output2": 1.0}
        model.compile('adam', loss=losses, loss_weights=lossWeights)
        
        print("x.shape", np.asarray(encode_input).shape)  #x.shape (2,  9)
        
 
        #x=[np.array(encode_input * 1)]
        #y=[np.array(target_errors * 1),np.array(target_LB * 1)]
       
        #print("x.shape", np.asarray(x).shape)  #x.shape (2, 2048, 9)
        
        ####  Split the data set into train and test_model
        x=np.array(encode_input)
        y=list(zip(np.array(target_errors), np.array(target_LB)))


        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

        y_train = list(zip(*y_train))
        y_train[0] = np.array(y_train[0])
        y_train[1] = np.array(y_train[1])

        
        y_test = list(zip(*y_test))
        y_test[0] = np.array(y_test[0])
        y_test[1] = np.array(y_test[1])
               
        
        saveTestTrainData("train_models/x_train_500.npy", x_train)
        saveTestTrainData("test_models/x_test_500.npy", x_test)
        saveTestTrainData("train_models/y_train0_500.npy", y_train[0])#target
        saveTestTrainData("train_models/y_train1_500.npy", y_train[1])#lb
        saveTestTrainData("test_models/y_test0_500.npy", y_test[0])
        saveTestTrainData("test_models/y_test1_500.npy", y_test[1])
    
        ####
        
        model.fit(
            x=x_train,
            y=y_train,
            epochs=500, #100 200 500
            batch_size=32,
        )
        model.save("test_model.h5")
        saveDictionary(source_token_dict, 'source_token_dict.pickle')
        
        model, source_token_dict = load("test_model.h5")

        # Predict
        x_train_loaded = loadTestTrainData("train_models/x_train_500.npy") 
        y_train_loaded = [  loadTestTrainData("train_models/y_train0_500.npy"),
                          loadTestTrainData("train_models/y_train1_500.npy") ]

        x_test_loaded =  loadTestTrainData("test_models/x_test_500.npy") 
        y_test_loaded = [ loadTestTrainData("test_models/y_test0_500.npy"),
                         loadTestTrainData("test_models/y_test1_500.npy")]
        
        out1, out2 = tfr.decode(
            model,
            #encode_input,
            x_test_loaded,
            max_len=source_max_len
        )
        print("out1: ", out1)
        #print("target_errors: ",target_errors) 
        print("target_errors: ", y_test_loaded[0])        
        print("out2: ", out2)
        #print("target_LB: ", target_LB)
        print("target_LB: ", y_test_loaded[1])

In [9]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [10]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [11]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [12]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [13]:
def load(model_name):
        import sys
        sys.path.append('Perfomer_local/keras_position_wise_feed_forward')
        sys.path.append('Perfomer_local/tensorflow_fast_attention')
        sys.path.append('Perfomer_local/keras_performer')
        #sys.path.append('/Perfomer_local/keras_pos_embed')

        from keras_performer import performer
        from tensorflow import keras
        from keras_embed_sim import EmbeddingRet, EmbeddingSim
        from keras_pos_embd import TrigPosEmbedding
        from tensorflow_fast_attention.fast_attention import  Attention, SelfAttention
        from keras_position_wise_feed_forward.feed_forward import FeedForward  

        co = performer.get_custom_objects()

        model = keras.models.load_model(model_name, custom_objects= co)
        source_token_dict = loadDictionary( 'source_token_dict.pickle')
       # t = loadDictionary(target_token_dict, 'target_token_dict.pickle')
       # t_inv = loadDictionary(target_token_dict_inv, 'target_token_dict_inv.pickle')
        return model, source_token_dict,

In [14]:
x=TestTranslate()
x.test_translate()

i am here: 
AAAA:  (1, 359, 36)
aaaa:  <class 'list'>
XXXX:  359
YYYY:  <class 'int'>
ZZZZ:  359
XXXX2:  359
<class 'int'>
In get_model: encoder_input:  (None, 2919)
In get_model: encoder_embed:  (None, 2919, 32)
Start Warpping................................
OOOO: KerasTensor(type_spec=TensorSpec(shape=(None, 2919, 32), dtype=tf.float32, name=None), name='Encoder-Embedding/add_1:0', description="created by layer 'Encoder-Embedding'")
embed_dim: 32
head_num: 4
dropout_rate: 0.05
masked: False
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 4, 8)
Key: XXXXXXX  (None, 2919, 4, 8)
Value: XXXXXXX  (None, 2919, 4, 8)
dim:  8
query:  Tensor("self_attention_1/query/einsum/Einsum:0", shape=(None, 2919, 4, 8), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 16)
Start Warpping............................

Epoch 1/500
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 4, 8)
Key: XXXXXXX  (None, 2919, 4, 8)
Value: XXXXXXX  (None, 2919, 4, 8)
dim:  8
query:  Tensor("model/self_attention_1/query/einsum/Einsum:0", shape=(None, 2919, 4, 8), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 16)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 4, 8)
Key: XXXXXXX  (None, 2919, 4, 8)
Value: XXXXXXX  (None, 2919, 4, 8)
dim:  8
query:  Tensor("model/self_attention_3/query/einsum/Einsum:0", shape=(None, 2919, 4, 8), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 16)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXX

11/11 [==============================] - 42s 4s/step - loss: 1877.0334 - error_feed_forward_output1_loss: 1835.1645 - error_feed_forward_output2_loss: 41.8690
Epoch 28/500
11/11 [==============================] - 41s 4s/step - loss: 2053.4226 - error_feed_forward_output1_loss: 2005.6737 - error_feed_forward_output2_loss: 47.7490
Epoch 29/500
11/11 [==============================] - 41s 4s/step - loss: 2872.8029 - error_feed_forward_output1_loss: 2827.1388 - error_feed_forward_output2_loss: 45.6643
Epoch 30/500
11/11 [==============================] - 41s 4s/step - loss: 2838.3054 - error_feed_forward_output1_loss: 2797.4866 - error_feed_forward_output2_loss: 40.8187
Epoch 31/500
11/11 [==============================] - 41s 4s/step - loss: 2566.6928 - error_feed_forward_output1_loss: 2526.5107 - error_feed_forward_output2_loss: 40.1819
Epoch 32/500
11/11 [==============================] - 41s 4s/step - loss: 3095.9396 - error_feed_forward_output1_loss: 3059.5776 - error_feed_forward_out

11/11 [==============================] - 42s 4s/step - loss: 16086.1509 - error_feed_forward_output1_loss: 16043.5025 - error_feed_forward_output2_loss: 42.6495
Epoch 75/500
11/11 [==============================] - 43s 4s/step - loss: 19657.9458 - error_feed_forward_output1_loss: 19612.3332 - error_feed_forward_output2_loss: 45.6110
Epoch 76/500
11/11 [==============================] - 41s 4s/step - loss: 16566.8029 - error_feed_forward_output1_loss: 16522.7329 - error_feed_forward_output2_loss: 44.0695
Epoch 77/500
11/11 [==============================] - 41s 4s/step - loss: 18079.2078 - error_feed_forward_output1_loss: 18033.2472 - error_feed_forward_output2_loss: 45.9604
Epoch 78/500
11/11 [==============================] - 42s 4s/step - loss: 26801.7686 - error_feed_forward_output1_loss: 26756.3836 - error_feed_forward_output2_loss: 45.3834
Epoch 79/500
11/11 [==============================] - 41s 4s/step - loss: 21488.6649 - error_feed_forward_output1_loss: 21446.2080 - error_feed

11/11 [==============================] - 38s 3s/step - loss: 47866.7764 - error_feed_forward_output1_loss: 47826.6589 - error_feed_forward_output2_loss: 40.1158
Epoch 122/500
11/11 [==============================] - 38s 3s/step - loss: 54371.9746 - error_feed_forward_output1_loss: 54326.5104 - error_feed_forward_output2_loss: 45.4608
Epoch 123/500
11/11 [==============================] - 38s 3s/step - loss: 48311.2402 - error_feed_forward_output1_loss: 48264.5212 - error_feed_forward_output2_loss: 46.7171
Epoch 124/500
11/11 [==============================] - 38s 3s/step - loss: 46707.7852 - error_feed_forward_output1_loss: 46665.2604 - error_feed_forward_output2_loss: 42.5236
Epoch 125/500
11/11 [==============================] - 38s 3s/step - loss: 47364.9541 - error_feed_forward_output1_loss: 47326.0498 - error_feed_forward_output2_loss: 38.9044
Epoch 126/500
11/11 [==============================] - 38s 3s/step - loss: 42822.2181 - error_feed_forward_output1_loss: 42777.0443 - error

...

11/11 [==============================] - 38s 3s/step - loss: 402703.0130 - error_feed_forward_output1_loss: 402663.1797 - error_feed_forward_output2_loss: 39.8598
Epoch 398/500
11/11 [==============================] - 38s 3s/step - loss: 368862.9479 - error_feed_forward_output1_loss: 368815.0807 - error_feed_forward_output2_loss: 47.8505
Epoch 399/500
11/11 [==============================] - 38s 3s/step - loss: 509760.4844 - error_feed_forward_output1_loss: 509716.4089 - error_feed_forward_output2_loss: 44.0632
Epoch 400/500
11/11 [==============================] - 38s 3s/step - loss: 427463.2552 - error_feed_forward_output1_loss: 427418.6667 - error_feed_forward_output2_loss: 44.6342
Epoch 401/500
11/11 [==============================] - 38s 3s/step - loss: 315660.2161 - error_feed_forward_output1_loss: 315613.3047 - error_feed_forward_output2_loss: 46.9089
Epoch 402/500
11/11 [==============================] - 38s 3s/step - loss: 358505.0495 - error_feed_forward_output1_loss: 358456.

11/11 [==============================] - 38s 3s/step - loss: 406326.6901 - error_feed_forward_output1_loss: 406285.4531 - error_feed_forward_output2_loss: 41.2404
Epoch 444/500
11/11 [==============================] - 38s 3s/step - loss: 322118.2422 - error_feed_forward_output1_loss: 322073.2031 - error_feed_forward_output2_loss: 45.0551
Epoch 445/500
11/11 [==============================] - 38s 3s/step - loss: 312332.7357 - error_feed_forward_output1_loss: 312294.4375 - error_feed_forward_output2_loss: 38.2985
Epoch 446/500
11/11 [==============================] - 38s 3s/step - loss: 366616.7760 - error_feed_forward_output1_loss: 366577.1536 - error_feed_forward_output2_loss: 39.6296
Epoch 447/500
11/11 [==============================] - 38s 3s/step - loss: 441345.6927 - error_feed_forward_output1_loss: 441302.4896 - error_feed_forward_output2_loss: 43.2087
Epoch 448/500
11/11 [==============================] - 38s 3s/step - loss: 389751.2812 - error_feed_forward_output1_loss: 389705.

11/11 [==============================] - 38s 3s/step - loss: 477522.1172 - error_feed_forward_output1_loss: 477480.9844 - error_feed_forward_output2_loss: 41.1327
Epoch 490/500
11/11 [==============================] - 38s 3s/step - loss: 481003.9453 - error_feed_forward_output1_loss: 480958.7839 - error_feed_forward_output2_loss: 45.1677
Epoch 491/500
11/11 [==============================] - 38s 3s/step - loss: 375807.6172 - error_feed_forward_output1_loss: 375768.3359 - error_feed_forward_output2_loss: 39.2747
Epoch 492/500
11/11 [==============================] - 38s 3s/step - loss: 391636.4531 - error_feed_forward_output1_loss: 391588.8229 - error_feed_forward_output2_loss: 47.6342
Epoch 493/500
11/11 [==============================] - 38s 3s/step - loss: 412479.2526 - error_feed_forward_output1_loss: 412432.5677 - error_feed_forward_output2_loss: 46.7045
Epoch 494/500
11/11 [==============================] - 38s 3s/step - loss: 510235.0833 - error_feed_forward_output1_loss: 510192.

C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 4, 8)
Key: XXXXXXX  (None, 2919, 4, 8)
Value: XXXXXXX  (None, 2919, 4, 8)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 8)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 8)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 4, 8)
Key: XXXXXXX  (None, 2919, 4, 8)
Value: XXXXXXX  (None, 2919, 4, 8)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 8)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 8)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 4, 8)
Key: XXXXXXX  (None, 2919, 4, 8)
Value: XXXXXXX  (None, 2919, 4, 8)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 4, 8)
KS OOOOOOOOOOOOO <class 'tens